# VGG
Visual Geometry Group实验室。

## VGG块
连续使用数个相同的padding为1，size为3 * 3的卷积层，后接上stride为2，size为2 * 2的最大池化层。

卷积层保持输入的高和宽不变，而池化层对其减半。

## 为何选择小卷积核？
对于给定（保证相同）感受野，采用堆积的小卷积核优于采用大的卷积核，因此可以加深网络深度保证学习更复杂的模式，代价还比较小。

VGG中3个3 * 3卷积核代替7 * 7卷积核，2个3 * 3卷积核代替5 * 5卷积核。

In [1]:
import time
import torch
from torch import nn, optim
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def vgg_block(num_convs, in_channels, out_channels):
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    # 这里会使宽高减半
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*blk)

In [3]:
conv_arch = ((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
# 经过5个vgg_block，宽高会减半5次，变成 224 / 32 = 7
fc_features = 512 * 7 * 7
fc_hidden_units = 4096

In [4]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096, net=nn.Sequential()):
    for i, (num_convs,  in_channels, out_channels) in enumerate(conv_arch):
        net.add_module('vgg_block_' + str(i+1), vgg_block(num_convs, in_channels, out_channels))
    net.add_module('fc', nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(fc_features, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, 10)
    ))
    return net

In [5]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)

# 获取子模块和一级子模块的名字，named_modules会返回所有子模块，包括子模块的名字
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

vgg_block_1 output shape:  torch.Size([1, 64, 112, 112])
vgg_block_2 output shape:  torch.Size([1, 128, 56, 56])
vgg_block_3 output shape:  torch.Size([1, 256, 28, 28])
vgg_block_4 output shape:  torch.Size([1, 512, 14, 14])
vgg_block_5 output shape:  torch.Size([1, 512, 7, 7])
fc output shape:  torch.Size([1, 10])


# 数据获取和训练模型

In [7]:
ratio = 8
small_conv_arch = [(1, 1, 64 // ratio), (1, 64 // ratio, 128 // ratio), (2, 128 // ratio, 256 // ratio), (2, 256 // ratio, 512 // ratio), (2, 512 // ratio, 512 // ratio)]
net = vgg(small_conv_arch, fc_features // ratio, fc_hidden_units // ratio)
print(net)

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
batch_size = 64
root = '~/Datasets'
train_iter, test_iter = d2l.load_data_fashion_mnist2(batch_size, resize=224, root=root)

In [10]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_cuda_cpu(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cpu
epoch  1, loss  2.2998, train acc  0.094, test acc  0.100, time  156.0 sec
epoch  1, loss  2.3036, train acc  0.078, test acc  0.100, time  307.0 sec
epoch  1, loss  2.3048, train acc  0.078, test acc  0.100, time  444.5 sec
epoch  1, loss  2.3028, train acc  0.078, test acc  0.100, time  585.6 sec
epoch  1, loss  2.3004, train acc  0.097, test acc  0.100, time  722.5 sec
epoch  1, loss  2.3032, train acc  0.091, test acc  0.100, time  860.7 sec
epoch  1, loss  2.3037, train acc  0.094, test acc  0.100, time  998.3 sec
epoch  1, loss  2.3039, train acc  0.096, test acc  0.100, time  1136.1 sec
epoch  1, loss  2.3048, train acc  0.094, test acc  0.100, time  1273.8 sec
epoch  1, loss  2.3052, train acc  0.094, test acc  0.100, time  1412.7 sec
epoch  1, loss  2.3054, train acc  0.095, test acc  0.100, time  1552.1 sec
epoch  1, loss  2.3053, train acc  0.098, test acc  0.100, time  1689.2 sec
epoch  1, loss  2.3042, train acc  0.097, test acc  0.100, time  1827.8 sec
ep

epoch  1, loss  2.2369, train acc  0.132, test acc  0.530, time  16117.0 sec
epoch  1, loss  2.2294, train acc  0.135, test acc  0.510, time  16256.5 sec
epoch  1, loss  2.2202, train acc  0.138, test acc  0.492, time  16405.8 sec
epoch  1, loss  2.2121, train acc  0.140, test acc  0.493, time  16562.2 sec
epoch  1, loss  2.2017, train acc  0.145, test acc  0.489, time  16718.3 sec
epoch  1, loss  2.1951, train acc  0.148, test acc  0.519, time  16861.0 sec
epoch  1, loss  2.1900, train acc  0.149, test acc  0.536, time  17001.2 sec
epoch  1, loss  2.1834, train acc  0.152, test acc  0.558, time  17142.6 sec
epoch  1, loss  2.1756, train acc  0.155, test acc  0.519, time  17286.3 sec
epoch  1, loss  2.1693, train acc  0.157, test acc  0.528, time  17430.9 sec
epoch  1, loss  2.1599, train acc  0.161, test acc  0.522, time  17572.2 sec
epoch  1, loss  2.1514, train acc  0.164, test acc  0.530, time  17740.7 sec
epoch  1, loss  2.1418, train acc  0.167, test acc  0.533, time  17885.7 sec

KeyboardInterrupt: 